# Collaborative Filtering Recommendation system

### Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

The data for this project is currently stored as two seperate csv files. I will use pandas to load them as two seperate dataframes and combine them by the "book id" column

In [6]:
ratings_file = "ratings.csv"
ratings_df = pd.read_csv(ratings_file)

books_file = "books.csv"
books_df = pd.read_csv(books_file)

combined_df = pd.merge(ratings_df, books_df, on="book_id")
combined_df.head()

,book_id,user_id,rating,id,best_book_id,work_id,books_count,isbn,isbn13,authors,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,314,5,27,1,41335427,275,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823,1785676,27520,7308,21516,136333,459028,1161491,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...
1,1,439,3,27,1,41335427,275,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823,1785676,27520,7308,21516,136333,459028,1161491,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...
2,1,588,5,27,1,41335427,275,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823,1785676,27520,7308,21516,136333,459028,1161491,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...
3,1,1169,4,27,1,41335427,275,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823,1785676,27520,7308,21516,136333,459028,1161491,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...
4,1,1185,4,27,1,41335427,275,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPré",...,1678823,1785676,27520,7308,21516,136333,459028,1161491,https://images.gr-assets.com/books/1361039191m...,https://images.gr-assets.com/books/1361039191s...


The next step is to pivot the table so that the book names are the columns and the rows are the users indivdual book ratings. In this step I also give any books that a user has not reviewed a socre of 0.

In [10]:
rating_utility_matrix = combined_df.pivot_table(values="rating", index="user_id", columns="title", fill_value=0)
rating_utility_matrix.head()

title,'Salem's Lot,"'Tis (Frank McCourt, #2)",1421: The Year China Discovered America,1776,1984,A Bend in the River,A Bend in the Road,A Brief History of Time,A Briefer History of Time,A Case of Need,...,"Women in Love (Brangwen Family, #2)",World War Z: An Oral History of the Zombie War,"World Without End (The Kingsbridge Series, #2)",Wuthering Heights,"Xenocide (Ender's Saga, #3)",Year of Wonders,You Shall Know Our Velocity!,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zodiac,number9dream
user_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0


In [11]:
rating_utility_matrix.shape

(28906, 812)

Here we can see there are 28,906 different customers. To find a more generalized look at customers reviews I'm going to use the TruncatedSVD iimport from sklearn. 

In [12]:
X = rating_utility_matrix.T
SVD = TruncatedSVD(n_components=30)
transposed_matrix = SVD.fit_transform(X)
transposed_matrix.shape

(812, 30)

Printing the shape of the transposed matrix shows that the original 28,906 rows fort the cusomers have now been compressed in to just 30.

The last step of this project is to make a function to recommend books based upon the input of a book title

In [13]:
corr_matrix = np.corrcoef(transposed_matrix)

In [42]:
def recommend(title,similarity):
    books = rating_utility_matrix.columns
    books_list = list(books)
    book_index = books_list.index(title)
    corr_title = corr_matrix[book_index]
    recommendations = list(books[(corr_title < 1.0) & (corr_title > similarity)])
    return recommendations

In [46]:
title = "Wuthering Heights"  #Change this to alter the starting book 
similarity = 0.8             #Change this to alter the amount of recommedations

print(recommend(title, similarity))

['Cane River', 'Night (The Night Trilogy #1)', 'The Power and the Glory', 'Wuthering Heights']


# Conclusion

In this note book I took the book and customer data from goodread and combined them to create on larger datafram that I them used sklearns SVD to reduce the di